# **ZMSI_Lab_9**
Generowanie danych dla pretrenowanych modeli YOLO

$\color{red}{Uwaga:}$

1- proszę pamiętać o zmianie słowa ***Album*** w nazwie pliku i uzupełnieniu swoich danych powyżej tego pola;

2- kluczowe polecenia proszę opatrzyć komentarzem;

3- wnioski, pod zadaniem, wpisujemy w polu tekstowym.

In [ ]:
# Podpisywanie pracy
NN = input("Podaj Imię i Nazwisko: ")
ALBUM = input("Podaj numer albumu: ")

Podaj Imię i Nazwisko: Kacper Gąsowski
Podaj numer albumu: 154845


In [ ]:
# Ocena pracy. Proszę nie zmieniać komórek z kodem pomocniczym do oceny.
punkty = []
max = int(60)

### **Zad 9.1.** Przygotuj środowisko Colaboratory do trenowania modelu YOLO 8.



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Instalacja biblioteki Ultralytics, która zawiera YOLOv8
!pip install ultralytics roboflow opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
# Import niezbędnych bibliotek
import os
import cv2
import random
import numpy as np
from roboflow import Roboflow
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

# Tworzenie katalogów dla danych, jeśli jeszcze ich nie ma
os.makedirs('content/drive/MyDrive/KUL/Laby9/images', exist_ok=True)
os.makedirs('content/drive/MyDrive/KUL/Laby9/labels', exist_ok=True)

print("Środowisko przygotowane!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Środowisko przygotowane!


In [ ]:
# Maksymalna wartość punktów za Zad 9.1: 5
punkty.append(5)

In [ ]:
!pip install roboflow


### **Zad 9.2** Dla obrazów ok. 50 obrazów wygeneruj dane treningowe, testowe i walidacyjne dla modelu YOLO w wersji 8 dla zadania detekcji i lokalizacji obiektów.

1- Obrazy wyszukaj w repozytorium Roboflow. Wybierz 2 klasy różniących się obiektów.

2- Etykietowanie obrazów wykonaj automatycznie (oferowane wsparcie CNN).

3- Następnie zweryfikuj i popraw etykietowanie (błędy mogą pojawić się głównie w etykietach klas).
2- Dane powinny być podzielone na odpowiednie zbiory w proporcjach: treningowe
80%, testowe 10%, walidacyjne 10%. Docelowy rozmiar obrazów powinien wynosić: szerokość 450 px, wysokość 600 px. Do obrazów powinien być dodany 2% szumu.

3- Sposób wykonania pokazany jest na filmie z wykładu.

4- Dataset pobierz do katalogu '/content'.


In [ ]:
from roboflow import Roboflow

# Klasa 1: Raccoons
rf = Roboflow(api_key="NvNjReBF50pyrorn4lNQ")
project1 = rf.workspace("prova-4sazq").project("raccoons-490gk")
version1 = project1.version(1)
dataset1 = version1.download("yolov8")

# Klasa 2: Lions
rf = Roboflow(api_key="NvNjReBF50pyrorn4lNQ")
project = rf.workspace("wild-animal-detection-hyeyj").project("lion-8nmfb")
version = project.version(1)
dataset = version.download("yolov8")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Raccoons-1 in yolov8:: 100%|██████████| 214/214 [00:00<00:00, 3397.87it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Lion-1 in yolov8:: 100%|██████████| 157/157 [00:00<00:00, 4863.81it/s]


In [ ]:
# Instalacja biblioteki Roboflow
!pip install roboflow

# Import bibliotek
from roboflow import Roboflow
import os
import shutil
from PIL import Image
import numpy as np

# Pobranie danych dla klasy 1: Raccoons
rf = Roboflow(api_key="NvNjReBF50pyrorn4lNQ")
project1 = rf.workspace("prova-4sazq").project("raccoons-490gk")
version1 = project1.version(1)
dataset1 = version1.download("yolov8")  # Obiekt Dataset

# Pobranie danych dla klasy 2: Lions
project2 = rf.workspace("wild-animal-detection-hyeyj").project("lion-8nmfb")
version2 = project2.version(1)
dataset2 = version2.download("yolov8")

# Pobranie ścieżek do katalogów
dataset1_path = dataset1.location
dataset2_path = dataset2.location

# Funkcja do łączenia danych
def merge_datasets(source_path, destination_path):
    for dataset_type in ['train', 'valid', 'test']:
        for folder in ['images', 'labels']:
            src = os.path.join(source_path, dataset_type, folder)
            dst = os.path.join(destination_path, dataset_type, folder)
            os.makedirs(dst, exist_ok=True)
            if os.path.exists(src):
                for file in os.listdir(src):
                    shutil.copy(os.path.join(src, file), dst)

# Łączenie danych
merge_datasets(dataset2_path, dataset1_path)
print("Dane zostały połączone!")

# Funkcja do skalowania obrazów i dodawania szumu
def process_images(image_folder):
    for img_file in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_file)
        if img_file.endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(img_path).resize((450, 600))  # Skalowanie do 450x600
            np_img = np.array(img)

            # Dodanie 2% szumu
            noise = np.random.normal(0, 0.02 * 255, np_img.shape).astype(np.int16)
            noisy_img = np.clip(np_img + noise, 0, 255).astype(np.uint8)

            # Zapis przetworzonego obrazu
            noisy_img = Image.fromarray(noisy_img)
            noisy_img.save(img_path)

# Przetwarzanie obrazów w zbiorach
for dataset_type in ['train', 'valid', 'test']:
    process_images(f"{dataset1_path}/{dataset_type}/images")

print("Obrazy zostały przeskalowane i dodano szum!")

# Przygotowanie pliku data.yaml
yaml_content = f"""
train: {dataset1_path}/train/images
val: {dataset1_path}/valid/images
test: {dataset1_path}/test/images

nc: 2
names: ['raccoon', 'lion']
"""

# Zapis pliku data.yaml
yaml_path = os.path.join(dataset1_path, 'data.yaml')
with open(yaml_path, 'w') as yaml_file:
    yaml_file.write(yaml_content)

print("Plik data.yaml został utworzony!")


loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
Dane zostały połączone!
Obrazy zostały przeskalowane i dodano szum!
Plik data.yaml został utworzony!


In [ ]:
# Maksymalna wartość punktów za Zad 9.2: 15
punkty.append(15)

### **Zad 9.3.** Dotrenuj model YOLO w wersji 8 dla zadania detekcji i lokalizacji obiektów na pobrabranych z Roboflow danych

In [ ]:
# Maksymalna wartość punktów za Zad 9.3: 10
punkty.append(10)

In [ ]:
from ultralytics import YOLO

# Ścieżka do pliku data.yaml
data_yaml_path = '/content/Raccoons-1/data.yaml'

# Inicjalizacja modelu YOLOv8 (pretrenowanego na COCO)
model = YOLO('yolov8n.pt')

# Trenowanie modelu
model.train(data=data_yaml_path, epochs=50, imgsz=640, batch=16, device=0)


100%|██████████| 6.25M/6.25M [00:00<00:00, 312MB/s]


Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Raccoons-1/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

100%|██████████| 755k/755k [00:00<00:00, 104MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 390MB/s]


...

                   all         46         80      0.543      0.588      0.558      0.289



50 epochs completed in 0.046 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.37it/s]


                   all         46         80      0.676      0.574        0.6      0.302
               raccoon         46         80      0.676      0.574        0.6      0.302
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ca6436a1000>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### **Zad 9.4.** Pliki z metrykami procesu uczenia i walidacji przekopiuj do osobnego katalogu, skompresuj, nadaj mu nazwę *ZMSI_L9_Album.zip* i umieść razem z notatnikiem w chmurze.

In [ ]:
import shutil

# Ścieżka do katalogu wynikowego treningu YOLO
source_path = '/content/runs/detect/train'
destination_path = '/content/ZMSI_L9_Metrics'

# Skopiowanie katalogu
shutil.copytree(source_path, destination_path)
print("Pliki metryk zostały skopiowane.")


Pliki metryk zostały skopiowane.


In [ ]:
shutil.make_archive('/content/ZMSI_L9_Album', 'zip', '/content/ZMSI_L9_Metrics')
print("Pliki zostały skompresowane.")


Pliki zostały skompresowane.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

shutil.move('/content/ZMSI_L9_Album.zip', '/content/drive/MyDrive/KUL/ZMSI_L9_Album.zip')
print("Pliki zostały przeniesione na chmurę.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Pliki zostały przeniesione na chmurę.


In [ ]:
# Maksymalna wartość punktów za Zad 9.4: 5
punkty.append(5)

### **Zad 9.5.** Dotrenuj model YOLO w wersji 8 dla jednego z pozostałych zadań rozpoznawania obiektów na pobrabranych z Roboflow danych

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="NvNjReBF50pyrorn4lNQ")
project = rf.workspace("final-year-386zb").project("car_models-izfw0-osk0c-mqdpo-uib7l")
version = project.version(1)
dataset = version.download("yolov8")



loading Roboflow workspace...
loading Roboflow project...


In [ ]:
# Sprawdzenie lokalizacji pobranych danych
dataset_location = dataset.location
print("Dane zostały pobrane do:", dataset_location)

# Sprawdzenie, czy plik 'data.yaml' istnieje
yaml_path = os.path.join(dataset_location, 'data.yaml')
print("Ścieżka do pliku data.yaml:", yaml_path)

Dane zostały pobrane do: /content/car_models-1
Ścieżka do pliku data.yaml: /content/car_models-1/data.yaml


In [ ]:
data_yaml_path = '/content/car_models-1/data.yaml'

# Inicjalizacja i trening modelu
model = YOLO('yolov8n.pt')
model.train(data=data_yaml_path, epochs=50, imgsz=640, batch=16, device=0)


Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/car_models-1/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

train: Scanning /content/car_models-1/train/labels... 793 images, 0 backgrounds, 0 corrupt: 100%|██████████| 793/793 [00:00<00:00, 1690.76it/s]

train: New cache created: /content/car_models-1/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/car_models-1/valid/labels... 160 images, 0 backgrounds, 0 corrupt: 100%|██████████| 160/160 [00:00<00:00, 801.17it/s]

val: New cache created: /content/car_models-1/valid/labels.cache


...

                   all        160        532      0.933      0.936      0.939       0.84



50 epochs completed in 0.251 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.3MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  1.71it/s]


                   all        160        532      0.952      0.933      0.941      0.841
                     0        143        156      0.993      0.941      0.979      0.825
                     1         56         71      0.986      0.978      0.988      0.865
                     2         48         48      0.996          1      0.995      0.952
                     3         48         48      0.979      0.983      0.995      0.902
                     4         48         48      0.996          1      0.995      0.928
                     5         48         48      0.997          1      0.995      0.916
                     6         48         48      0.996          1      0.995      0.927
                     7         48         48      0.996          1      0.995       0.93
                   car         10         10      0.841          1      0.995      0.778
                  cars          5          7      0.739      0.429       0.48      0.387
Speed: 0.3ms preproce

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ca65cd6ef50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0

In [ ]:
# Maksymalna wartość punktów za Zad 9.5: 25
punkty.append(25)

In [ ]:
razem = sum(punkty)
razem_pr = razem/max
if razem_pr >= 0.90:
  ocena = 5
elif razem_pr >= 0.80:
  ocena = 4.5
elif razem_pr >= 0.70:
  ocena = 4
elif razem_pr >= 0.60:
  ocena = 3.5
elif razem_pr >= 0.50:
  ocena = 3
else:
  ocena = 2

print("Punktacja za kolejne zadania: ", punkty)
print("Suma punktów: ", razem)
print("Udział procentowy: ", razem_pr*100, "%")
print("Ocena: ", ocena)

Punktacja za kolejne zadania:  [5, 15, 10, 5, 25]
Suma punktów:  60
Udział procentowy:  100.0 %
Ocena:  5
